This code is based on:
http://machinelearningmastery.com/understanding-stateful-lstm-recurrent-neural-networks-python-keras/

# Import libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential, Model
from keras.layers import LSTM , Dense, Input
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences

%matplotlib inline

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
char_to_ind = dict((c,i) for i,c in enumerate(alphabet))
ind_to_char = dict((i,c) for i,c in enumerate(alphabet))

In [16]:
max_len = 10
dataX = []
dataY = []
X_size = 1000
for i in range(X_size):
    start = np.random.randint(0, len(alphabet) - max_len)
    end = np.random.randint(start, start+max_len)
    seq_in = alphabet[start:end+1]
    seq_out = alphabet[end+1]
    dataX.append([char_to_ind[char] for char in seq_in])
    dataY.append([char_to_ind[char] for char in seq_out])
    if i < 20:
        print(seq_in,"->",seq_out)

('LMN', '->', 'O')
('JKLMNO', '->', 'P')
('FGH', '->', 'I')
('JKLMN', '->', 'O')
('KLMNOP', '->', 'Q')
('H', '->', 'I')
('KLMNOPQRS', '->', 'T')
('GHIJKL', '->', 'M')
('ABCDEF', '->', 'G')
('KLMN', '->', 'O')
('ABCDEFGHIJ', '->', 'K')
('FGHIJKLM', '->', 'N')
('FGH', '->', 'I')
('DEFGHIJK', '->', 'L')
('M', '->', 'N')
('IJ', '->', 'K')
('NOP', '->', 'Q')
('HIJ', '->', 'K')
('CD', '->', 'E')
('EFGHIJ', '->', 'K')


In [4]:
len(dataX)

1000

In [5]:
X = pad_sequences(dataX, maxlen=max_len, dtype='float32')

In [6]:
X = np.reshape(X,[len(X), max_len, 1])

In [7]:
X = X / float(len(alphabet))

In [8]:
y = np_utils.to_categorical(dataY)

In [9]:
y.shape , X.shape

((1000, 26), (1000, 10, 1))

In [10]:
split = 0.7
trainX = X[:int(split*X.shape[0]) , : , :]
trainY = y[:int(split*X.shape[0]) , :]
testX = X[int(split*X.shape[0]): , : , :]
testY = y[int(split*X.shape[0]): , :]

In [12]:
model = Sequential()
model.add(LSTM(20, input_shape=(X.shape[1],X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, nb_epoch=1000, batch_size=50, verbose=2)

Epoch 1/1000
1s - loss: 3.2504 - acc: 0.0100
Epoch 2/1000
0s - loss: 3.2367 - acc: 0.0400
Epoch 3/1000
0s - loss: 3.2209 - acc: 0.0600
Epoch 4/1000
0s - loss: 3.2020 - acc: 0.0600
Epoch 5/1000
0s - loss: 3.1749 - acc: 0.0600
Epoch 6/1000
0s - loss: 3.1366 - acc: 0.0600
Epoch 7/1000
0s - loss: 3.0920 - acc: 0.0714
Epoch 8/1000
0s - loss: 3.0327 - acc: 0.1014
Epoch 9/1000
0s - loss: 2.9648 - acc: 0.1014
Epoch 10/1000
0s - loss: 2.8879 - acc: 0.1129
Epoch 11/1000
0s - loss: 2.8025 - acc: 0.1129
Epoch 12/1000
0s - loss: 2.7387 - acc: 0.1171
Epoch 13/1000
0s - loss: 2.6838 - acc: 0.1057
Epoch 14/1000
0s - loss: 2.6369 - acc: 0.1214
Epoch 15/1000
0s - loss: 2.6005 - acc: 0.1157
Epoch 16/1000
0s - loss: 2.5612 - acc: 0.1486
Epoch 17/1000
0s - loss: 2.5252 - acc: 0.1600
Epoch 18/1000
0s - loss: 2.4832 - acc: 0.1657
Epoch 19/1000
0s - loss: 2.4388 - acc: 0.1929
Epoch 20/1000
0s - loss: 2.3997 - acc: 0.1700
Epoch 21/1000
0s - loss: 2.3656 - acc: 0.1714
Epoch 22/1000
0s - loss: 2.3390 - acc: 0.17

In [13]:
outputs = model.evaluate(testX, testY, verbose=0)
print(model.metrics_names[0] + " is " + str(outputs[0]) + " and " + model.metrics_names[1] + " is " + str(outputs[1]*100))

loss is 0.283634632428 and acc is 93.3333333333


In [15]:
for j in range(10):
    random_n = np.random.randint(0, len(testX))
    item = testX[random_n]
    item = pad_sequences([item], maxlen=max_len)
    char_series = np.reshape(item, [1, max_len, 1])
    char_series = char_series / float(len(alphabet))
    y_pred = model.predict(char_series)
    ind = np.argmax(y_pred)
    char_pred = ind_to_char[ind]
    char_seq = [ind_to_char[j] for j in dataX[random_n]]
    print(str(char_seq) + "->" + str(char_pred))

['J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q']->C
['F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']->C
['B', 'C', 'D']->C
['O']->C
['A', 'B', 'C', 'D', 'E']->C
['D', 'E']->C
['D', 'E']->C
['A', 'B', 'C', 'D', 'E', 'F']->C
['E', 'F', 'G', 'H', 'I', 'J']->C
['H', 'I', 'J', 'K']->C
